# HDX ACAPS data import

In [ ]:
import pandas as pd
import xlrd
import csv
import pycountry
import datetime
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
# papermill parameters
output_folder = '../output/'

In [ ]:
response = requests.get('https://data.humdata.org/dataset/e1a91ae0-292d-4434-bc75-bf863d4608ba')
soup = BeautifulSoup(response.text)
data = soup.find("a", {'href':re.compile("xls"), "title": "Download"})['href']

In [ ]:
df = pd.read_excel(f"https://data.humdata.org{data}", sheet_name="Database")

In [ ]:
def get_country(row):
    country = pycountry.countries.get(alpha_3=row.ISO)
    
    if country:
        return country.alpha_2
    else:
        return ""

In [ ]:
df["ISO3166_1"] = df.apply(get_country, axis=1)

In [ ]:
df = df.drop(columns=["ISO", "REGION", "Alternative source"], axis=1)

In [ ]:
df["TARGETED_POP_GROUP"] = df["TARGETED_POP_GROUP"].str.rstrip()
df["NON_COMPLIANCE"] = df["NON_COMPLIANCE"].str.capitalize()

In [ ]:
df = df.rename(columns={
    "COUNTRY": "COUNTRY_STATE",
    "ADMIN_LEVEL_NAME": "ADMIN_2"
})

In [ ]:
df["LAST_UPDATED_DATE"] = datetime.datetime.now()

In [ ]:
df.to_csv(path_or_buf=output_folder + "HDX_ACAPS.csv", quoting = csv.QUOTE_NONNUMERIC,
          columns=[
              "COUNTRY_STATE", "ADMIN_2", "REGION", "CATEGORY", "MEASURE",
              "TARGETED_POP_GROUP", "COMMENTS", "NON_COMPLIANCE",
              "DATE_IMPLEMENTED", "SOURCE", "SOURCE_TYPE", "LINK",
              "ENTRY_DATE", "ISO3166_1", "LAST_UPDATED_DATE"
          ], index=False)